In [1]:
%pip install transformers
%pip install sklearn
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import gc
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tc2code = pd.read_json('./data.jsonl',lines=True)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(tc2code.head())

                                            testCode  \
0  /*\n * This file was automatically generated b...   
1  /*\n * This file was automatically generated b...   
2  /*\n * This file was automatically generated b...   
3  /*\n * This file was automatically generated b...   
4  /*\n * This file was automatically generated b...   

                                          sourceCode  
0  package macaw.util;\n\nimport macaw.system.Mac...  
1  package macaw.util;\n\nimport macaw.system.Use...  
2  package macaw.util;\n\nimport java.util.regex....  
3  package macaw.util;\n\nimport java.awt.event.C...  
4  package macaw.util;\n\nimport javax.swing.*;\n...  


In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base').to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [7]:
def RobertaTokenizerForSeq2Seq(x):
  return tokenizer.batch_encode_plus(x,max_length=1800,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

In [8]:
def RobertaTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=514,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

In [9]:
print(RobertaTokenizerForSeq2Seq(tc2code['sourceCode'][0]))

{'input_ids': tensor([[    0,   642,     2,  ...,     1,     1,     1],
        [    0,   102,     2,  ...,     1,     1,     1],
        [    0,   438,     2,  ...,     1,     1,     1],
        ...,
        [    0, 24303,     2,  ...,     1,     1,     1],
        [    0, 50118,     2,  ...,     1,     1,     1],
        [    0, 50118,     2,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [10]:
X_train, X_test, y_train, y_test = train_test_split(tc2code['testCode'], tc2code['sourceCode'], test_size=0.2, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.1, shuffle=False)

In [11]:
X_train_tensors=RobertaTokenizerForSeq2SeqTest(X_train.to_list())
X_test_tensors=RobertaTokenizerForSeq2SeqTest(X_test.to_list())
X_val_tensors=RobertaTokenizerForSeq2SeqTest(X_val.to_list())

In [12]:
y_train_tensors=RobertaTokenizerForSeq2Seq(y_train.to_list())
y_test_tensors=RobertaTokenizerForSeq2Seq(y_test.to_list())
y_val_tensors=RobertaTokenizerForSeq2Seq(y_val.to_list())

In [13]:
input_ids_train = X_train_tensors['input_ids']
attention_masks_train = X_train_tensors['attention_mask']
labels_train = y_train_tensors['input_ids']

#validation set
input_ids_val = X_val_tensors['input_ids']
attention_masks_val = X_val_tensors['attention_mask']
labels_val = y_val_tensors['input_ids']


input_ids_test = X_test_tensors['input_ids']
attention_masks_test = X_test_tensors['attention_mask']
labels_test = y_test_tensors['input_ids']
expected_source_code=y_test.to_list()

In [14]:
gc.collect()

0

In [15]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            labels_val)

dataset_test = TensorDataset(input_ids_test,
                             attention_masks_test,
                             labels_test)

In [16]:


batch_size = 1

#train set
dataloader_train = DataLoader(dataset_train,
                              sampler = RandomSampler(dataset_train),
                              batch_size = batch_size)

#validation set
dataloader_val = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = batch_size)

dataloader_test = DataLoader(dataset_val,
                              sampler = RandomSampler(dataset_val),
                              batch_size = batch_size)

In [17]:


optimizer = torch.optim.AdamW(model.parameters(),
                 lr = 1e-5,
                 eps = 1e-8)
                 
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = len(dataloader_train)*epochs)

In [18]:
def train_model(model):
    save_epoch=5
    validation_loss_epochs=[]
    training_loss_epochs=[]
    torch.cuda.empty_cache()
    for epoch in tqdm(range(1, epochs+1)):
        
        model.train()
        
        loss_train_total = 0
        
        progress_bar = tqdm(dataloader_train, 
                            desc = 'Epoch {:1d}'.format(epoch), 
                            leave = False, 
                            disable = False)
        
        for batch in progress_bar:
            
            model.zero_grad() #set gradient to 0
        
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids': batch[0], 
                      'attention_mask': batch[1], 
                      'labels': batch[2]}
            
            outputs = model(**inputs) #unpack the dict straight into inputs
            
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

        if epoch%save_epoch==0:    
          torch.save(model.state_dict(), f'Models/ CodeT5_ft_epoch{epoch}.model')
        
        tqdm.write('\n Epoch {epoch}')
        
        loss_train_ave = loss_train_total / len(dataloader_train)
        tqdm.write('Training loss: {loss_train_avg}')
        
        val_loss = evaluate(dataloader_val,model)
        validation_loss_epochs.append(val_loss)
        training_loss_epochs.append(loss_train_ave)
        
        tqdm.write(f'Validation loss: {val_loss}')
        return validation_loss_epochs,training_loss_epochs,model

In [19]:
def evaluate(dataloader_val,model):

    #evaluation mode 
    model.eval()
    
    #tracking variables
    loss_val_total = 0
    
    for batch in tqdm(dataloader_val):
        
        #load into GPU
        batch = tuple(b.to(device) for b in batch)
        
        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        loss_val_total += loss.item()

    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
            
    return loss_val_avg

In [20]:
torch.cuda.empty_cache()

In [21]:
val_losses, training_losses, model=train_model(model)

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemmStridedBatched( handle, opa, opb, m, n, k, &alpha, a, lda, stridea, b, ldb, strideb, &beta, c, ldc, stridec, num_batches)`